In [1]:
from k8stencil import patterns

In [9]:
pod_affinity_labels = {
    "app.kubernetes.io/name": "thanos-query",
}

selector_labels = {
    "app.kubernetes.io/component": "query-layer",
    "app.kubernetes.io/instance": "thanos-query",
    **pod_affinity_labels,
}

labels = {
    "app.kubernetes.io/version": "v0.31.0",
    **selector_labels,
}

ports = dict(
    grpc=10901,
    http=9090,
)

service = dict(
    apiVersion="v1",
    kind="Service",
    metadata=dict(
        labels=labels,
        name="thanos-query",
        namespace="thanos",
    ),
    spec=dict(
        ports=list(patterns.ports_to_service_ports(ports)),
        selector=selector_labels,
    ),
)



In [10]:
args = dict(
    grpc_address=patterns.address(ports, "grpc"),
    http_address=patterns.address(ports, "http"),
    log=dict(
        level="info",
        format="logfmt",
    ),
    query=dict(
        replica_label=["prometheus_replica", "rule_replica"],
        auto_downsampling=None,
    ),
    endpoint=[
        patterns.service_dnssrv(svc, "grpc", 10901)
        for svc in ["thanos-store", "thanos-receive-ingestor-default"]
    ]
)

env_fields = dict(
       host_ip_address="status.hostIP",
)

containers = [
    dict(
        args=["query"] + list(patterns.args_to_list(args)),
        env=list(patterns.env_fields_to_env(env_fields)),
        ports=list(patterns.ports_to_container_ports(ports)),
        readinessProbe=patterns.http_probe(ports, 4, 30, "ready"),
        livenessProbe=patterns.http_probe(ports, 20, 5, "healthy"),
        image="quay.io/thanos/thanos:v0.31.0",
        name="thanos-query",
        resources={},
        terminationMessagePolicy="FallbackToLogsOnError",
    ),
]



In [12]:
deployment_spec_template_spec = dict(
    affinity=patterns.get_pod_anti_affinity(pod_affinity_labels, "thanos"),
    nodeSelector={"kubernetes.io/os": "linux"},
    securityContext={
        'fsGroup': 65534,
        'runAsGroup': 65532,
        'runAsNonRoot': True,
        'runAsUser': 65534,
        'seccompProfile': {'type': 'RuntimeDefault'}
    },
    containers=containers,
    serviceAccountName="thanos-query",
    terminationGracePeriodSeconds=120,
)

In [15]:
deployment_spec = dict(
  replicas=1,
  selector=dict(
      matchLabels=labels,
  ),
  template=dict(
    metadata=dict(labels=labels),
    spec=deployment_spec_template_spec,
  ),
)

In [18]:
deployment = dict(
    apiVersion="apps/v1",
    kind="Deployment",
    metadata=dict(
        labels=labels,
        name=labels["app.kubernetes.io/instance"],
        namespace="thanos",
    ),
    spec=deployment_spec,
)

In [19]:
deployment

{'apiVersion': 'apps/v1',
 'kind': 'Deployment',
 'metadata': {'labels': {'app.kubernetes.io/version': 'v0.31.0',
   'app.kubernetes.io/component': 'query-layer',
   'app.kubernetes.io/instance': 'thanos-query',
   'app.kubernetes.io/name': 'thanos-query'},
  'name': 'thanos-query',
  'namespace': 'thanos'},
 'spec': {'replicas': 1,
  'selector': {'matchLabels': {'app.kubernetes.io/version': 'v0.31.0',
    'app.kubernetes.io/component': 'query-layer',
    'app.kubernetes.io/instance': 'thanos-query',
    'app.kubernetes.io/name': 'thanos-query'}},
  'template': {'metadata': {'labels': {'app.kubernetes.io/version': 'v0.31.0',
     'app.kubernetes.io/component': 'query-layer',
     'app.kubernetes.io/instance': 'thanos-query',
     'app.kubernetes.io/name': 'thanos-query'}},
   'spec': {'affinity': {'podAntiAffinity': {'preferredDuringSchedulingIgnoredDuringExecution': [{'podAffinityTerm': {'labelSelector': {'matchExpressions': [{'key': 'app.kubernetes.io/name',
            'operator': '

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    app.kubernetes.io/component: query-layer
    app.kubernetes.io/instance: thanos-query
    app.kubernetes.io/name: thanos-query
    app.kubernetes.io/version: v0.31.0
  name: thanos-query
  namespace: thanos
spec:
  replicas: 1
  selector:
    matchLabels:
      app.kubernetes.io/component: query-layer
      app.kubernetes.io/instance: thanos-query
      app.kubernetes.io/name: thanos-query
  template:
    metadata:
      labels:
        app.kubernetes.io/component: query-layer
        app.kubernetes.io/instance: thanos-query
        app.kubernetes.io/name: thanos-query
        app.kubernetes.io/version: v0.31.0
    spec:
      affinity:
        podAntiAffinity:
          preferredDuringSchedulingIgnoredDuringExecution:
          - podAffinityTerm:
              labelSelector:
                matchExpressions:
                - key: app.kubernetes.io/name
                  operator: In
                  values:
                  - thanos-query
              namespaces:
              - thanos
              topologyKey: kubernetes.io/hostname
            weight: 100
      containers:
      - args:
        - query
        - --grpc-address=0.0.0.0:10901
        - --http-address=0.0.0.0:9090
        - --log.level=info
        - --log.format=logfmt
        - --query.replica-label=prometheus_replica
        - --query.replica-label=rule_replica
        - --endpoint=dnssrv+_grpc._tcp.thanos-store.thanos.svc.cluster.local:10901
        - --endpoint=dnssrv+_grpc._tcp.thanos-receive-ingestor-default.thanos.svc.cluster.local:10901
        - --query.auto-downsampling
        env:
        - name: HOST_IP_ADDRESS
          valueFrom:
            fieldRef:
              fieldPath: status.hostIP
        image: quay.io/thanos/thanos:v0.31.0
        imagePullPolicy: IfNotPresent
        livenessProbe:
          failureThreshold: 4
          httpGet:
            path: /-/healthy
            port: 9090
            scheme: HTTP
          periodSeconds: 30
        name: thanos-query
        ports:
        - containerPort: 10901
          name: grpc
        - containerPort: 9090
          name: http
        readinessProbe:
          failureThreshold: 20
          httpGet:
            path: /-/ready
            port: 9090
            scheme: HTTP
          periodSeconds: 5
        resources: {}
        securityContext:
          allowPrivilegeEscalation: false
          capabilities:
            drop:
            - ALL
          readOnlyRootFilesystem: true
          runAsGroup: 65532
          runAsNonRoot: true
          runAsUser: 65534
          seccompProfile:
            type: RuntimeDefault
        terminationMessagePolicy: FallbackToLogsOnError
      nodeSelector:
        kubernetes.io/os: linux
      securityContext:
        fsGroup: 65534
        runAsGroup: 65532
        runAsNonRoot: true
        runAsUser: 65534
        seccompProfile:
          type: RuntimeDefault
      serviceAccountName: thanos-query
      terminationGracePeriodSeconds: 120
```